In [1]:
import openpyxl
from openpyxl.utils import column_index_from_string, get_column_letter

import openpyxl
from openpyxl.utils import column_index_from_string
import openpyxl
import pandas as pd


### Pre-traitement

### Input et Output entrée

#### Output

In [2]:
# Specify the path to your Excel file
excel_file_path = './../Donnees/Databrut.xlsx'

# Load the workbook
workbook = openpyxl.load_workbook(excel_file_path)

# Select the active sheet
sheet = workbook.active

# Specify the column letters you want to select
selected_columns_letters = ["K","O", "P", "Q"]

# Convert letters to column indices
selected_columns_indices = [column_index_from_string(letter) for letter in selected_columns_letters]
# Use the indices to select columns


# Create a dictionary to store the selected columns
stored_data = {}

# Define column names (you can customize these)
column_names = ["Water_out", "Power_out", "Power_in", "COP"]

# Iterate over the selected columns and store data in the dictionary
for col_index, column_name in zip(selected_columns_indices, column_names):
    # Access the column using the column index
    selected_column = sheet[get_column_letter(col_index)]
    # Store column data in the dictionary
    stored_data[column_name] = [cell.value for cell in selected_column]

# Now, 'stored_data' is a dictionary where keys are column names and values are lists of column data

# Close the workbook
workbook.close()

In [3]:
# Convert stored_data to a Pandas DataFrame
df = pd.DataFrame(stored_data)

# Convert the entire DataFrame to numeric values
df_numeric = df.apply(pd.to_numeric, errors='coerce')

# Remove rows that contain non-numeric values
df_output = df_numeric.dropna()

# Display the cleaned DataFrame
print(df_output)

     Water_out    Power_out     Power_in       COP
4    35.098729  8628.775284  2022.298343  4.279641
5    35.170939  8589.444308  2044.149171  4.214619
6    35.029724  7641.797307  1748.723757  4.383122
7    35.118453  6873.783742  1477.259669  4.667061
8    35.050442  6035.440249  1248.497238  4.848746
..         ...          ...          ...       ...
129  54.203023  3519.253648  2367.003754  1.486797
130  54.241624  3545.644394  2262.551720  1.567100
131  53.182450  2757.048670  1684.523486  1.636694
132  51.688952  1626.207401  1171.696004  1.387909
133  49.027759   138.371200   735.992197  0.188006

[77 rows x 4 columns]


In [4]:
# export l'ouput en fichier Excel
Outputpath = './../Donnees/Outputdata.xlsx'
df_output.to_excel(Outputpath, index=False)

#### Input

In [5]:
# Specify the path to your Excel file
excel_file_path = './../Donnees/Databrut.xlsx'

# Load the workbook
workbook = openpyxl.load_workbook(excel_file_path)

# Select the active sheet
sheet = workbook.active

# Specify the column letters you want to select
selected_columns_letters = ["I", "J","L"]

# Convert letters to column indices
selected_columns_indices = [column_index_from_string(letter) for letter in selected_columns_letters]

# Use the indices to select columns
selected_columns = sheet.iter_cols(min_col=min(selected_columns_indices), max_col=max(selected_columns_indices), values_only=True)

# Create a dictionary to store the selected columns
stored_data = {}

# Define column names (you can customize these)
column_names =["Frequency", "Water_in", "Flow" ]

# Iterate over the selected columns and store data in the dictionary
for column, column_name in zip(selected_columns, column_names):
    stored_data[column_name] = list(column)

# Now, 'stored_data' is a dictionary where keys are column names

# Close the workbook
workbook.close()

# Ajout de la colonne T_air
NbA12W35,NbA12W45 = 9,9
NbA7W35,NbA7W45,NbA7W55 = 13,12,4
NbA2W35,NbA2W45,NbA2W55  = 5,5,5
NbAmoins7W35,NbAmoins7W45,NbAmoins7W55 = 5,5,5

NbA12 = NbA12W35+NbA12W45
NbA7 = NbA7W35+NbA7W45+NbA7W55
NbA2 = NbA2W35+NbA2W45+NbA2W55
NbAmoins7 = NbAmoins7W35+NbAmoins7W45+NbAmoins7W55

values = [12]*NbA12 + [7]*NbA7 + [2]*NbA2 + [-7]*NbAmoins7


In [6]:
# Convert stored_data to a Pandas DataFrame
df = pd.DataFrame(stored_data)

# Convert the entire DataFrame to numeric values
df_numeric = df.apply(pd.to_numeric, errors='coerce')

# Remove rows that contain non-numeric values
df_input = df_numeric.dropna()

# Assign values to the "T_air" column 
df_input['T_air'] = values

# Display the cleaned DataFrame
print(df_input)

     Frequency   Water_in       Flow  T_air
4        105.6  30.112818  35.098729     12
5         98.4  30.139006  35.170939     12
6         88.2  30.081215  35.029724     12
7         78.0  30.209724  35.118453     12
8         68.4  30.096077  35.050442     12
..         ...        ...        ...    ...
129      105.6  49.644142  54.203023     -7
130       88.2  49.644146  54.241624     -7
131       68.4  49.610203  53.182450     -7
132       48.0  49.582365  51.688952     -7
133       28.8  48.848730  49.027759     -7

[77 rows x 4 columns]


/tmp/ipykernel_6258/3459895846.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_input['T_air'] = values


In [7]:
# export l'input en fichier Excel
Inputpath = './../Donnees/Inputdata.xlsx'
df_input.to_excel(Inputpath, index=False)

### Class

In [8]:
# Import the needed (FMPy) packages
from fmpy import dump, read_model_description, extract
from fmpy.fmi2 import FMU2Slave
from fmpy.util import plot_result #, download_test_file
import numpy as np
#import shutil

In [67]:
class CustomSimulator:
    def __init__(self, fmu_path):
        self.fmu_path = fmu_path
        self.model_description = read_model_description(fmu_path)
        self.unzipdir = extract(fmu_path)
        self.modelIdentifier = self.model_description.coSimulation.modelIdentifier
        self.guid = self.model_description.guid
        self.vrs = {
            var.name: var.valueReference for var in self.model_description.modelVariables}
        
        self.input_variables = [
        var for var in self.model_description.modelVariables if var.causality == 'input']

    def simulate(self,Frequency,Water_in,Flow,T_air):
        start_time = 0.0
        stop_time = 1500.0
        step_size = 1

        fmu = FMU2Slave(guid=self.guid, unzipDirectory=self.unzipdir,
                        modelIdentifier=self.modelIdentifier, instanceName='instance1')

        fmu.instantiate()

        start_values = {
            'u_compressorFrequency': 88.2,     # Hz
            'u_VFlow': 1.33 / 3600,            # m3/s
            'u_T_watrer_in': 303.15,           # K
            'u_T_air': 285.15,                 # K

            'x_areaLeakage': 5e-7,             # m², valeur dans [1e-8 ; 1e-6]
            'x_areaSuctionValve': 5e-4,        # m², valeur dans [2e-6 ; 1e-3]
            'x_areaDischargeValve': 5e-5,      # m², valeur dans [1e-4 ; 1e-7]
            'x_relativeDeadSpace': 0.05,       # valeur dans [0.001 ; 0.1]
            'x_driveEfficiency': 0.9           # valeur dans [0.85 ; 0.98]
        }

        start_values['u_compressorFrequency'] = Frequency
        start_values['u_VFlow'] = Flow
        start_values['u_T_watrer_in'] = Water_in
        start_values['u_T_air'] = T_air
 

        fmu.setupExperiment(startTime=start_time)


        for var_name, start_value in start_values.items():
            fmu.setReal([self.vrs[var_name]], [start_value])


        fmu.enterInitializationMode()
        fmu.exitInitializationMode()


        # print('Simulating...')
        time = np.arange(start_time, stop_time, step_size)
        result = []

        for t in time:
            fmu.doStep(currentCommunicationPoint=t,
                    communicationStepSize=step_size)
            
            # Retrieve the outputs
            COP_Freq, Power_in, Power_out,Water_out = fmu.getReal(
                [self.vrs['y_COP'], self.vrs['y_heatPower'], self.vrs['y_elecPower'], self.vrs['y_T_out']])
            result.append((t, COP_Freq, Power_in, Power_out,Water_out))

        fmu.terminate()
        fmu.freeInstance()

        result = np.array(result, dtype=np.dtype([('time', np.float64), (
            'y_COP', np.float64), ('Power_in', np.float64), ('Power_out', np.float64)]))

        return result

    def run_simulations(self, inputs):
        for index, row in inputs.iterrows():
            self.simulate(Frequency=row['Frequency'], Water_in=row['Water_in'],
                          Flow=row['Flow'],T_air=row['T_air'])


In [169]:
# Assuming df is a DataFrame with simulation parameters
simulator = CustomSimulator('HPFMU_20_Linux.fmu')

Frequency,Water_in,Flow,T_air  = df_input.iloc[0, :]
result = simulator.simulate(Frequency,Water_in,Flow,T_air)
# result[-1]

# simulator.run_simulations(df_input)

TypeError: CustomSimulator.simulate() takes 1 positional argument but 5 were given